# Fitting a gaussian curve with an MLP

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from fa import *
from train import *
from utils import *

In [ ]:
n = 1000
x = np.linspace(-2, 2, n).reshape(-1, 1)
y = 0.5*np.exp(-(x*1.5)**2) - 0.3 + 0.02 * np.random.randn(*x.shape)
plt.plot(x, y, 'ro');

### Train with and without feedback alignment layers

In [ ]:
dataset = tensor_data(x, y)

net1 = MLP(1, 5, 1)
loss1 = train_net(net1, dataset, 50, optimizer="Adam")['loss']

net2 = MLP(1, 5, 5, 1, linearity=FALinear)
loss2 = train_net(net2, dataset, 50, optimizer="Adam")['loss']

# plot training curves
plt.figure(0)
plt.title("training curves")
plt.plot(loss1);
plt.plot(loss2);
plt.legend(['BP','FA']);

# plot solutions
plt.figure(1)
plt.title("fits")
y2 = fromT(net1(toT(x)))
y3 = fromT(net2(toT(x)))
plt.plot(x, y, 'ro');
plt.plot(x, y2)
plt.plot(x, y3);
plt.legend(['data', 'BP','FA']);